Importing the necessary packages

In [ ]:
import numpy as np
import math
from functools import reduce
from copy import deepcopy

****Necessary functions of the quantum simulator**

The basis of the qubit system

In [ ]:
qubit01 = {0: np.array([1 , 0 ]),1: np.array([0 , 1])}

Tensor of the qubit system

In [ ]:
#translation of the qubit system into a common vector
#input(list) - list of qubits(0 or 1)
def tensorqubit(list):
  q = reduce((lambda x,y: np.kron(x,y)), list)
  return q

Hadamard + NOT + INOT (You can still add a complex one and still lead NOTs)

In [ ]:
H = np.array([[1/math.sqrt(2),1/math.sqrt(2)],[1/math.sqrt(2),-1/math.sqrt(2)]])
X = np.array([[0,1],[1,0]])
I = np.array([[1,0],[0,1]])

i the number of the qubit in a bit record of size n

In [ ]:
# Translation of the i qubit by count into a bit record of size n
# Input (n) is the number of qubits, (i) is the number of qubits 
# int output bit entry
def inttobit(n, i):
    return list(map(int, list(format(i, "0{0}b".format(n)))))

CNOT function for the n qubit system (controlling, controlled, n qubit){for toffoli, you still need to write a loop inside}

In [ ]:
# Finds the CNOT matrix in the n qubit system
# Input: (q1) is the number of the controlling qubit, (q2) is the number of the controlled qubit, (n) is the total qubit in the system  
# Output is a 4*4 matrix (so far without the complex part)
def CNOT(q1, q2, n):
  matrixCNOT = np.zeros((2 ** n, 2 ** n))
  for i in range(2 ** n):
    binary = inttobit(n, i)
    matrixCNOT[:, i] =reduce(np.kron, [
        [qubit01[k]] if j != q2 else qubit01[((binary[q1 - 1])^ (binary[q2 - 1]))] \
        for j, k in enumerate(binary, start=1)])
  return matrixCNOT

**DEUTSCH - JOZSA**

**New required features**

the function of the i step, but through the sheet (there were problems translating tuple, so I had to change the function a little)

In [ ]:
# Finds the matrix of unitary operators of the i-th step
# Input - A list of unitary operators in this step (List)
# The output is the matrix of such a step n**2 * n**2
def UnitStep(list):
  Step_i = reduce((lambda x,y: np.kron(x,y)), list)
  return Step_i

функция цепи, но через лист(возникли проблемы перевода tuple поэтому пришлось поменять чуть функцию)

In [ ]:
# Transformation of the qubit system after the gate chain
# # Input (Steps number) - the number of steps in the chain, (qubitsstart) - the initial state of the qubit,(list) - the list of the general transformation matrix at the i-th step
# Output - The state of the qubits after passing the chain (vector n**2)
def Chainqubit(qubitsstart, list):
  for i in range(len(list)):
    qubitsstart = list[i].dot(qubitsstart)
  return qubitsstart

Functions of the Deutsch-Joj algorithm (balanced and constant)

In [ ]:
def balanceOracul(n):

  randombalance = np.random.randint(1,2**n)
  randombalancebit = inttobit(n, randombalance)
  XNOTCHAIN = []
  CNOTCHAIN = []
  CHAINORACULE = []
  for i in range(n):
    if randombalancebit[i] == 1:
      XNOTCHAIN.append(X)
    else:
      XNOTCHAIN.append(I)
  XNOTCHAIN.append(I)
  CHAINORACULE.append(UnitStep(XNOTCHAIN))
  for j in range(n):
    CHAINORACULE.append(CNOT(n - j,n+1,n+1))
  
  CHAINORACULE.append(UnitStep(XNOTCHAIN))

  return CHAINORACULE

def constant(n):
  randomconst = np.random.randint(2)
  const=[]
  for i in range(n):
    const.append(I)
  if randomconst == 1:
    const.append(X)
  else:
    const.append(I)
  return UnitStep(const)
    


n hadamard gates

In [ ]:
def Hn(n):
  H_n = []
  for i  in range(n):
    H_n.append(H)
  return UnitStep(H_n)

Setting the number of qubits

In [ ]:
n = int(input())
m1 = np.array([1, 0])
m2 = np.array([0, 1])
systemQubit = []
for i in range(n):
  systemQubit.append(m1)
systemQubit.append(m2)
systemQubit

3


[array([1, 0]), array([1, 0]), array([1, 0]), array([0, 1])]

common vector qubit

In [ ]:
qubit = tensorqubit(systemQubit)

Sets the finished chain. For balanced - balance, for constant - const

In [ ]:
FullChain = []
FullChain.append(Hn(n+1))
indicator = input()
if indicator == 'balance':
  Oracul = balanceOracul(n)
  for i in range(2+n):
    FullChain.append(Oracul[i])
if indicator == 'const':
  Oracul = constant(n)
  FullChain.append(Oracul)
FullChain.append(np.kron(Hn(n),I))

balance


Outputs the result (balanced or constant){then you need to implement the black box in a more interesting way}

In [ ]:
o = Chainqubit(qubit, FullChain)
o = o*o
indexnonzero = np.where(o > 0.001)
print((bin(indexnonzero[0][0])[2:].zfill(n+1))[:n])
print((bin(indexnonzero[0][1])[2:].zfill(n+1))[:n])

111
111
[1, 1, 1, 0]
[1, 1, 1, 1]


If the first n qubits = 0 => constant

If the first n qubits = 1 => Balanced

**